In [8]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Boundary Polygons
coon_points = np.array([
    [[0, 0, 0], [1, 0, 1], [2, 0, 1], [3, 0, 1], [4, 0, 1]],
    [[0, 1, 0], [np.nan, np.nan, np.nan], [np.nan, np.nan, np.nan], [np.nan, np.nan, np.nan], [4, 1, 3]],
    [[0, 2, 0], [np.nan, np.nan, np.nan], [np.nan, np.nan, np.nan], [np.nan, np.nan, np.nan], [4, 2, 3]],
    [[0, 3, 0], [1, 3, 1], [2, 3, 1], [3, 3, 1], [4, 3, 1]]
])

m = 3
n = 4

# De Casteljau's algorithm function definition
def dca(c, r, i, t):
    if r == 0:
        return c[i]
    return (1 - t) * dca(c, r - 1, i, t) + t * dca(c, r - 1, i + 1, t)

def dca_part1(u, points):
    return np.array([dca(points[:, j], m, 0, u) for j in range(points.shape[1])])

def dca_part2(u, v, points):
    return dca(dca_part1(u, points), n, 0, v)

# Generate Intermediate points for Coons Patch
for i in range(1, m):
    for j in range(1, n):
        temp = (
            (1 - i / m) * coon_points[0, j] +
            (i / m) * coon_points[m, j] +
            (1 - j / n) * coon_points[i, 0] +
            (j / n) * coon_points[i, n] -
            np.dot(
                np.array([[1 - j / n, j / n]]),
                np.dot(
                    np.array([[1 - i / m], [i / m]]),
                    np.array([
                        [coon_points[0, 0], coon_points[0, n]],
                        [coon_points[m, 0], coon_points[m, n]]
                    ])
                ).reshape(2, 3)
            ).reshape(3)
        )
        coon_points[i, j] = temp

# Function to compute Gaussian curvature
def gaussian_curvature(u_val, v_val, h, coon_points):
    xu = (dca_part2(u_val + h, v_val, coon_points) - dca_part2(u_val - h, v_val, coon_points)) / (2 * h)
    xv = (dca_part2(u_val, v_val + h, coon_points) - dca_part2(u_val, v_val - h, coon_points)) / (2 * h)
    nxu = (dca_part2(u_val + 2 * h, v_val, coon_points) - 2 * dca_part2(u_val + h, v_val, coon_points) + dca_part2(u_val, v_val, coon_points)) / (h ** 2)
    nxv = (dca_part2(u_val, v_val + 2 * h, coon_points) - 2 * dca_part2(u_val, v_val + h, coon_points) + dca_part2(u_val, v_val, coon_points)) / (h ** 2)
    
    # Regularization
    epsilon = 1e-10
    F = np.linalg.det(np.array([[np.dot(xu, xu) + epsilon, np.dot(xu, xv)], [np.dot(xu, xv), np.dot(xv, xv) + epsilon]]))
    S = np.linalg.det(np.array([[np.dot(nxu, xu), np.dot(nxu, xv)], [np.dot(nxv, xu), np.dot(nxv, xv)]]))
    
    K = S / F
    return K

# Points to compute Gaussian curvature
points = [(0.25, 0.25), (0.25, 0.75), (0.5, 0.5), (0.75, 0.25), (0.75, 0.75)]

# Compute Gaussian curvature at specified points
curvatures = [gaussian_curvature(u, v, 0.01, coon_points) for u, v in points]

# Output Gaussian curvatures
print("Gaussian Curvatures at specified points:", curvatures)

# Function to draw u-partial and v-partial vectors
def draw_partial_vectors(points, coon_points):
    h = 0.01
    vectors = []
    for u, v in points:
        u_partial = (dca_part2(u + h, v, coon_points) - dca_part2(u, v, coon_points)) / h
        v_partial = (dca_part2(u, v + h, coon_points) - dca_part2(u, v, coon_points)) / h
        vectors.append((dca_part2(u, v, coon_points), u_partial, v_partial))
    return vectors

# Example usage
partial_vectors = draw_partial_vectors(points, coon_points)

# Plotting
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot control points
for i in range(coon_points.shape[0]):
    ax.plot(coon_points[i, :, 0], coon_points[i, :, 1], coon_points[i, :, 2], 'ro-')
for j in range(coon_points.shape[1]):
    ax.plot(coon_points[:, j, 0], coon_points[:, j, 1], coon_points[:, j, 2], 'ro-')

# Plot Bezier surface
u_vals = np.linspace(0, 1, 30)
v_vals = np.linspace(0, 1, 30)
U, V = np.meshgrid(u_vals, v_vals)
X, Y, Z = np.vectorize(lambda u, v: dca_part2(u, v, coon_points))(U, V)
ax.plot_surface(X, Y, Z, color='b', alpha=0.5)

# Plot partial vectors
for point, u_partial, v_partial in partial_vectors:
    ax.quiver(point[0], point[1], point[2], u_partial[0], u_partial[1], u_partial[2], color='r')
    ax.quiver(point[0], point[1], point[2], v_partial[0], v_partial[1], v_partial[2], color='b')

plt.show()

ValueError: shapes (2,1) and (2,2,3) not aligned: 1 (dim 1) != 2 (dim 1)